
#  Chainguard Threat Intelligence — Demonstration Notebook (Ethereum Sample)

This notebook uses your uploaded **`ethereum_sample_data.csv`** and walks through:

1. **Transaction input**
2. **Hashing** (salted SHA-256)
3. **Feature extraction** (value, fees, counts, z-score, temporal features)
4. **Risk scoring** (uses trained model if present; heuristic fallback otherwise)
5. **Classification & reporting** (Top‑100, Watchlist, Deep‑Dive)
6. **Rich Visualizations** for risk scores

> Run cells top-to-bottom. Outputs are saved next to the notebook so your Streamlit app can reuse them.


In [ ]:

import os
import pandas as pd
import numpy as np
import joblib
import hashlib

# Display options
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 160)

BASE_DIR = os.path.abspath('.')
DATA_PATH = r"C:\Users\pravin sharma\OneDrive\Documents\BugSlayers_ChainGuard\Data"  # uploaded dataset
MODEL_DIR = os.path.join(BASE_DIR, 'models')
MODEL_PATH = os.path.join(MODEL_DIR, 'chainguard_if_model.pkl')
SCALER_PATH = os.path.join(MODEL_DIR, 'chainguard_scaler.pkl'
# Output artifact paths (compatible with your app)
OUT_ALL_SCORED = os.path.join(BASE_DIR, 'all_scored_transactions.csv')
OUT_TOP100 = os.path.join(BASE_DIR, 'top_100_risky_transactions.csv')
OUT_WATCHLIST = os.path.join(BASE_DIR, 'watchlist_accounts.csv')
OUT_DEEPDIVE = os.path.join(BASE_DIR, 'watchlist_deep_dive_report.csv')

RISK_THRESHOLD = 95
WATCHLIST_SIZE = 50

print('BASE_DIR:', BASE_DIR)
print('DATA_PATH exists:', os.path.exists(DATA_PATH))


BASE_DIR: c:\Users\pravin sharma\Downloads
DATA_PATH exists: False


In [10]:

# --- Hashing & normalization ---

def secure_hash(address: str, salt: str = "chainguard_secret_key_2025") -> str:
    if pd.isna(address):
        return None
    return hashlib.sha256((str(address).strip().lower() + salt).encode()).hexdigest()


def normalize_hash(h: str) -> str:
    if h is None:
        return None
    return str(h).strip().lower()



## 1) Load Transactions
We load the uploaded **`ethereum_sample_data.csv`**. The notebook includes best‑effort header mapping for common variants.


In [11]:

# Best-effort header mapping (your file already matches these names)
rename_map = {
    'timestamp': 'block_timestamp',
    'time': 'block_timestamp',
    'tx_hash': 'transaction_hash',
    'hash': 'transaction_hash',
    'from': 'from_address',
    'to': 'to_address',
    'value_wei': 'value',
    'amount': 'value'
}

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Input CSV not found at {DATA_PATH}.")

# Read CSV
# Note: values like '0E-9' are valid scientific notation (zero); pd.to_numeric will coerce as needed later
raw = pd.read_csv(DATA_PATH)

# Apply header mapping only if needed
for k, v in rename_map.items():
    if k in raw.columns and v not in raw.columns:
        raw[v] = raw[k]

print(f"Loaded {len(raw)} rows")
raw.head(10)


FileNotFoundError: Input CSV not found at c:\Users\pravin sharma\Downloads\ethereum_sample_data.csv.


## 2) Hashing & Feature Extraction
We compute the features used by the risk model / heuristic:
- `value_eth`, `tx_fee`, `hour`
- counts per wallet (`from_address_hashed_tx_count`, `to_address_hashed_tx_count`)
- `value_z_score` (sender-side)
- `time_since_last_sender_tx`


In [ ]:

# Coerce types safely
raw['block_timestamp'] = pd.to_datetime(raw.get('block_timestamp'), errors='coerce', utc=True)
raw['value'] = pd.to_numeric(raw.get('value'), errors='coerce')
raw['gas'] = pd.to_numeric(raw.get('gas'), errors='coerce')
raw['gas_price'] = pd.to_numeric(raw.get('gas_price'), errors='coerce')

# Normalize addresses and hash (privacy-preserving)
raw['from_address'] = raw.get('from_address', '').astype(str).str.strip().str.lower()
raw['to_address'] = raw.get('to_address', '').astype(str).str.strip().str.lower()
raw['from_address_hashed'] = raw['from_address'].apply(secure_hash)
raw['to_address_hashed'] = raw['to_address'].apply(secure_hash)

# Derived features
raw = raw.sort_values('block_timestamp').reset_index(drop=True)
raw['value_eth'] = raw['value'] / (10**18)
raw['tx_fee'] = raw['gas'] * raw['gas_price']
raw['hour'] = raw['block_timestamp'].dt.hour

# Counts per hashed address
for col in ['from_address_hashed', 'to_address_hashed']:
    raw[f'{col}_tx_count'] = raw.groupby(col)['block_timestamp'].transform('count')

# Sender value history: mean/std → z-score
hist = raw.groupby('from_address_hashed')['value_eth'].agg(['mean', 'std'])
hist.columns = ['sender_mean', 'sender_std']
df_proc = raw.merge(hist, left_on='from_address_hashed', right_index=True, how='left')

# Z-score with safe NaN/Inf handling
df_proc['value_z_score'] = ((df_proc['value_eth'] - df_proc['sender_mean']) / df_proc['sender_std'])
df_proc['value_z_score'] = np.nan_to_num(df_proc['value_z_score'], nan=0.0, posinf=0.0, neginf=0.0)

# Time since last sender transaction
df_proc['time_since_last_sender_tx'] = (
    df_proc.groupby('from_address_hashed')['block_timestamp']
           .diff()
           .dt.total_seconds()
           .fillna(0)
)

print('Feature matrix shape:', df_proc.shape)
df_proc.head(10)



## 3) Risk Scoring
If a trained model & scaler are present in `./models/`, they will be used. Otherwise we compute a heuristic risk score that still maps to **0..100**.


In [ ]:

features = [
    'value_eth', 'tx_fee', 'hour',
    'from_address_hashed_tx_count', 'to_address_hashed_tx_count',
    'value_z_score', 'gas', 'gas_price', 'time_since_last_sender_tx'
]

use_model = os.path.exists(MODEL_PATH) and os.path.exists(SCALER_PATH)
print('Model available:', use_model)

if use_model:
    try:
        model = joblib.load(MODEL_PATH)
        scaler = joblib.load(SCALER_PATH)
        X = df_proc[features].fillna(0).values
        Xs = scaler.transform(X)
        raw_score = model.decision_function(Xs)
        min_s, max_s = np.min(raw_score), np.max(raw_score)
        denom = (max_s - min_s) if (max_s - min_s) != 0 else 1.0
        risk = 100 * (1 - (raw_score - min_s) / denom)
        df_proc['risk_score'] = np.round(risk, 2)
    except Exception as e:
        print('Model inference failed. Falling back to heuristic:', e)
        use_model = False

if not use_model:
    # Heuristic: combine |z| and normalized fee
    z = df_proc['value_z_score'].fillna(0).to_numpy()
    fee = df_proc['tx_fee'].fillna(0).to_numpy()
    fee_norm = (fee - fee.min()) / (fee.max() - fee.min() + 1e-9)
    raw_h = 0.6 * np.abs(z) + 0.4 * fee_norm
    raw_norm = (raw_h - raw_h.min()) / (raw_h.max() - raw_h.min() + 1e-9)
    df_proc['risk_score'] = np.round(100 * raw_norm, 2)

# Preview
cols_preview = ['transaction_hash','from_address_hashed','to_address_hashed','value_eth','risk_score','block_timestamp']
df_proc[cols_preview].head(10)



## 4) Classification & Reports
We produce the standard artifacts used by the Streamlit app:
- `all_scored_transactions.csv`
- `top_100_risky_transactions.csv` (ordered columns)
- `watchlist_accounts.csv`
- `watchlist_deep_dive_report.csv`


In [ ]:

# Save ALL scored data
df_proc.to_csv(OUT_ALL_SCORED, index=False)
print('Saved:', OUT_ALL_SCORED)

# Top-100 risky transactions (ordered per UI requirement)
report_cols = ['transaction_hash', 'value_eth', 'risk_score', 'block_timestamp', 'from_address_hashed']
top_100 = df_proc.sort_values(by='risk_score', ascending=False).head(100)
top_100[report_cols].to_csv(OUT_TOP100, index=False)
print('Saved:', OUT_TOP100)

# Watchlist (count high-risk events across sender & receiver)
df_high = df_proc[df_proc['risk_score'] >= RISK_THRESHOLD].copy()
sender_counts = df_high['from_address_hashed'].value_counts().reset_index()
sender_counts.columns = ['wallet_hash', 'risky_tx_count']
receiver_counts = df_high['to_address_hashed'].value_counts().reset_index()
receiver_counts.columns = ['wallet_hash', 'risky_tx_count']

wallet_risk = pd.concat([sender_counts, receiver_counts]).groupby('wallet_hash').sum().reset_index()
watchlist = wallet_risk.sort_values('risky_tx_count', ascending=False).head(WATCHLIST_SIZE)
watchlist.to_csv(OUT_WATCHLIST, index=False)
print('Saved:', OUT_WATCHLIST)

# Deep-Dive: all transactions where sender OR receiver is in watchlist
wl = watchlist['wallet_hash'].tolist()
deep = df_proc[(df_proc['from_address_hashed'].isin(wl)) | (df_proc['to_address_hashed'].isin(wl))].copy()
deep['direction'] = np.where(
    deep['from_address_hashed'].isin(wl), 'sent',
    np.where(deep['to_address_hashed'].isin(wl), 'received', 'other')
)

deep.to_csv(OUT_DEEPDIVE, index=False)
print('Saved:', OUT_DEEPDIVE)

# Quick preview of Top-100
top_100[['transaction_hash','risk_score']].head(10)



## 5) Rich Visualizations for Risk Scores
The following plots help explain risk distribution and drivers:
- **Histogram & KDE**
- **Box/Violin by hour of day**
- **Scatter: Value (ETH) vs Risk**
- **Time series: Risk over time**
- **Correlation heatmap**
- **Top wallets by avg risk**


In [ ]:

import matplotlib.pyplot as plt

risk = pd.to_numeric(df_proc['risk_score'], errors='coerce')
plt.figure(figsize=(7,4))
plt.hist(risk.dropna(), bins=30, color='#ef4444', alpha=0.85, edgecolor='white')
plt.title('Risk Score Histogram')
plt.xlabel('Risk Score')
plt.ylabel('Frequency')
plt.grid(True, alpha=0.25)
plt.show()


In [ ]:

# KDE (if seaborn available)
try:
    import seaborn as sns
    plt.figure(figsize=(7,4))
    sns.kdeplot(risk.dropna(), fill=True, color='#ef4444')
    plt.title('Risk Score Density (KDE)')
    plt.xlabel('Risk Score')
    plt.ylabel('Density')
    plt.grid(True, alpha=0.2)
    plt.show()
except Exception as e:
    print('Seaborn not available or KDE failed:', e)


In [ ]:

# Boxplot & Violin by hour
try:
    import seaborn as sns
    df_plot = df_proc[['hour','risk_score']].dropna()
    df_plot['hour'] = pd.to_numeric(df_plot['hour'], errors='coerce')

    plt.figure(figsize=(10,4))
    sns.boxplot(data=df_plot, x='hour', y='risk_score', palette='Reds')
    plt.title('Risk Score by Hour of Day (Boxplot)')
    plt.xlabel('Hour of Day')
    plt.ylabel('Risk Score')
    plt.grid(True, axis='y', alpha=0.2)
    plt.show()

    plt.figure(figsize=(10,4))
    sns.violinplot(data=df_plot, x='hour', y='risk_score', palette='Reds')
    plt.title('Risk Score by Hour of Day (Violin)')
    plt.xlabel('Hour of Day')
    plt.ylabel('Risk Score')
    plt.grid(True, axis='y', alpha=0.2)
    plt.show()
except Exception as e:
    print('Seaborn not available for box/violin:', e)


In [ ]:

# Scatter: Value (ETH) vs Risk
val = pd.to_numeric(df_proc['value_eth'], errors='coerce')
plt.figure(figsize=(7,5))
plt.scatter(val, risk, s=10, alpha=0.6, c=risk, cmap='Reds')
plt.title('Value (ETH) vs Risk')
plt.xlabel('Value (ETH)')
plt.ylabel('Risk Score')
plt.grid(True, alpha=0.25)
plt.colorbar(label='Risk Score')
plt.show()


In [ ]:

# Time series: Risk over time
ts = pd.to_datetime(df_proc['block_timestamp'], errors='coerce')
plt.figure(figsize=(10,4))
plt.plot(ts, risk, color='#ef4444', alpha=0.7)
plt.title('Risk Over Time')
plt.xlabel('Time')
plt.ylabel('Risk Score')
plt.grid(True, alpha=0.25)
plt.tight_layout()
plt.show()


In [ ]:

# Correlation heatmap
try:
    import seaborn as sns
    feat_cols = [
        'value_eth','tx_fee','hour',
        'from_address_hashed_tx_count','to_address_hashed_tx_count',
        'value_z_score','gas','gas_price','time_since_last_sender_tx','risk_score'
    ]
    df_corr = df_proc[feat_cols].copy().apply(pd.to_numeric, errors='coerce')
    corr = df_corr.corr()
    plt.figure(figsize=(9,7))
    sns.heatmap(corr, cmap='Reds', annot=True, fmt='.2f', square=True)
    plt.title('Feature Correlation Heatmap')
    plt.tight_layout()
    plt.show()
except Exception as e:
    print('Seaborn not available for heatmap:', e)


In [ ]:

# Top wallets by average risk (sender-side)
rank = df_proc.groupby('from_address_hashed')['risk_score'].mean().sort_values(ascending=False).head(10)
print('Top 10 wallets by average risk (sender-side):')
print(rank)



---
### Notes
- If trained model files are not present, heuristic fallback is applied automatically.
- The generated CSVs are compatible with your Streamlit dashboard.
- You can safely re-run sections to iterate quickly during demos.
